In [ ]:
import os
os.environ["THEANO_FLAGS"] = "device=gpu"
from tasks import check

In [ ]:
v = check(what="notebook", 
          filename="training/40mnist/model.pkl", 
          dataset="digits")

In [ ]:
capsule, data, layers, w, h, c = v
print(w, h, c)
print(layers.keys())
#print(w, h, c)
#print(data.X.min(), data.X.max())

## Hidden factors

In [ ]:
%matplotlib inline
from IPython.html.widgets import (interact, interactive, 
                                  IntSlider, FloatSlider,
                                  Button
                                  )
from IPython.display import display # Used to display widgets in the notebook

from IPython.html.widgets import *
from IPython.html import widgets
from lasagne import layers as L
import theano
import theano.tensor as T
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt

In [ ]:
use_examples = True# init/work with examples for dataset or not
nb = 1000 # nb of examples to consider in case use_examples is True
max_nb_sliders = 5
examples = data.X[0:nb]
examples = examples.reshape((examples.shape[0], c, w, h))
outputs = data.y[0:nb]
nb_outputs = data.output_dim


In [ ]:
rec = capsule.reconstruct(examples)
plt.imshow(rec[np.random.randint(0, rec.shape[0]), 0], cmap="gray", interpolation='bilinear')

In [ ]:
factor_name ="hidfactors"
x = T.tensor4()
y = T.matrix()
z = T.matrix()
encode = theano.function([x], L.get_output(layers[factor_name], x))

decode = theano.function([z, y], L.get_output(layers["output"], {layers[factor_name]: z, layers["y"]: y}))
latent_size = layers[factor_name].output_shape[1]
print("Latent size : {}".format(latent_size))

In [ ]:
z = encode(examples)

if z.shape[1] > max_nb_sliders:
    params = np.random.choice(z.shape[1],
                              size=10)
else:
    params = np.arange(z.shape[1])

boundaries = OrderedDict()
for p in (params):
    boundaries["{0}".format(p)] = FloatSlider(min=-1*z[:, p].std(),
                                              max=1*z[:, p].std(),
                                              step=0.01,
                                              value=0.)
d = 0
l = outputs[d]

def draw(**all_params):
    if use_examples is True:
        example = all_params["example"]
        del all_params["example"]
    label = all_params["label"]
    del all_params["label"]
    params = all_params
    
    if use_examples is True:
        z = encode(examples[example:example + 1])
    else:
        z = np.zeros((1, latent_size), dtype="float32")
        z[0, :] = 0
    
    y_ = np.zeros(nb_outputs, dtype='float32')
    y_[label] = 1.
    y_ = y_[np.newaxis, :]
        
    for k, v in params.items():
        z[0][int(k)] = v
    plt.imshow(decode(z, y_)[0, 0], cmap="gray", interpolation='bilinear')
    

p = dict()
p.update(boundaries)

label_selector = IntSlider(min=0,max=nb_outputs-1,step=1,value=l)
p["label"] = label_selector

if use_examples is True:
    example_selector = IntSlider(min=0,max=nb-1,step=1,value=d)
    p["example"] = example_selector

i = interact(**p)



def on_button_clicked(b):
    
    example = example_selector.get_state()["value"]
    z = encode(examples[example:example + 1])
    for p in params:
        w = boundaries["{0}".format(p)]
        state = w.get_state()
        state["value"] = z[0, int(p)]
        w.set_state(state)
        w.send_state(state)
        
    state = label_selector.get_state()
    state["value"] = outputs[example]
    label_selector.set_state(state)
    label_selector.send_state(state)
    
    
draw_i = i(draw)

if use_examples is True:
    button = widgets.Button(description="fit!")
    display(button)
    button.on_click(on_button_clicked)